In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader
import gradio as gr

In [ ]:
load_dotenv(override=True)

In [ ]:
openai = OpenAI(
    base_url=os.getenv("OPENROUTER_API_URL"), api_key=os.getenv("OPENROUTER_API_KEY")
)

In [ ]:
reader = PdfReader("me/resume.pdf")
resume = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        resume += text

resume

In [ ]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

summary

In [ ]:
name = "Rahul Dey"

In [ ]:
system_prompt = (
    f"You are acting as {name}. You are answering questions on {name}'s website, \
    particularly questions related to {name}'s career, background, skills and experience. \
    Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
    You are given a summary of {name}'s background and resume which you can use to answer questions. \
    Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
    If you don't know the answer, say so. "
)

system_prompt += f"\n\n## Summary: \n {summary}\n\n## Resume: \n{resume}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

print(system_prompt)

In [ ]:
def chat(message, history):
    messages = (
        [{"role": "system", "content": system_prompt}]
        + history
        + [{"role": "user", "content": message}]
    )
    response = openai.chat.completions.create(
        model="openai/gpt-oss-20b:free", messages=messages
    )
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat).launch()